In [ ]:
import tarfile
import boto3
import torchreid
import torch

## If import torchreid fails, please go back to README.md and ensure the "python setup.py develop" command was successful


In [ ]:
# Download the datasets prior to initializing the data manager
# For instructions on downloading the dataset, please follow the instructions @ https://kaiyangzhou.github.io/deep-person-reid/datasets.html#datasets 
# This example uses Market 1501 dataset and hte CUHK dataset
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources=['market1501', 'cuhk03'],
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=['random_flip', 'random_crop'])

In [ ]:
# You can list all the available models using the command 
# torchreid.models.show_avai_models()

# This example is using RESNET50 with ADAM optimizer
model = torchreid.models.build_model(
    name='resnet50',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)
model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [ ]:
# You can run the engine using 
# For more options, please visit https://kaiyangzhou.github.io/deep-person-reid/pkg/engine.html
engine.run(
    save_dir='log/resnet50',
    max_epoch=60,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

In [ ]:
# Now the model is ready to be exported for use in Panorama
# Specify the input (Batch Size=1, Channel=2, Heigth=256, Width=128)
inp = torch.rand([1, 3, 256, 128])
model.eval()
model_trace = torch.jit.trace(model.to('cuda'), inp.to('cuda'))
torch.jit.save(model_trace, 'model.pth')

with tarfile.open('model.tar.gz', 'w:gz') as f:
    f.add('model.pth')
f.close()

bucket = 'aws-panorama-<YOUR BUCKET NAME>'
desired_object_name = 'reid_model_v1.tar.gz'
s3 = boto3.resource('s3')
s3.meta.client.upload_file('model.tar.gz', bucket, desired_object_name)